In [11]:
from PIL import Image
import pytesseract
import argparse
import cv2
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import csv
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet 

In [54]:

# img=cv2.imread('/Users/realpranavc/REAL BITCAMP/Bitcamp-2023/images/ppweewee.jpeg')
img=cv2.imread('/Users/realpranavc/REAL BITCAMP/Bitcamp-2023/images/online-receipt-maker-and-design-food-templates_wo_Image.png')
# img=cv2.imread('/Users/realpranavc/REAL BITCAMP/Bitcamp-2023/images/online-receipt-maker-and-design-food-templates_wo_Image.png')
# image2 = cv2.normalize(image, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
# Apply the sharpening kernel to the image using filter2D
img = cv2.filter2D(img, -1, kernel)
# th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
#             cv2.THRESH_BINARY,11,2)

# cv2.imshow('Original image',image)
# cv2.imshow('Gray image', gray)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# resultimage = np.zeros((800, 800))
# normalizing the given image using normalize() function
# normalizedimage = cv2.normalize(gray,resultimage, 0, 100, cv2.NORM_MINMAX)


In [55]:
#convert it into text
text=(pytesseract.image_to_string(img)).lower()
vals = text.split('\n')
pattern = r'^(\d*)\s*([[a-zA-Z+|\s|\d]+)\:? (\$?(\d+\.*\d+))$'
pattern2 = r'^(\d*)\s*([[a-zA-Z+|\s|\d]+)\:? (\$?(\d+[\.|\:]*\d+))$'
prices= []
items= []
quantity= []
prices_no_dollar= []

amount= 0
tax= 0
subtotal= 0

done= True
for line in vals:
    # print( line)
    # print( re.match(pattern, line))
    max1= re.match(pattern, line)
    max2= re.match(pattern2, line)
    if max1:
        if float(max1.group(4)) <= 10000:
            if "amount" in (max1.group(2)).lower() or "total" in (max1.group(2)).lower():
                amount= float(max1.group(4))
                done= False
            elif "tax" in (max1.group(2)).lower():
                tax+= float(max1.group(4))
                done= False
            else:
                if done:
                    if max1.group(1) == "":
                        quantity.append(1)
                    else:
                        quantity.append(int(max1.group(1)))
                    items.append(max1.group(2))
                    prices.append(max1.group(3))
                    prices_no_dollar.append(max1.group(4))
    elif max2:
        sum1 = max2.group(4).replace(':','.')
        if float(sum1) <= 10000:
            if "amount" in (max2.group(2)).lower() or "total" in (max2.group(2)).lower():
                sum1 = max2.group(4).replace(':','.')
                amount= float(sum1)
                done= False
            elif "tax" in (max2.group(2)).lower():
                sum1 = max2.group(4).replace(':','.')
                tax+= float(sum1)
                done= False
            else:
                if done:
                    if max2.group(1) == "":
                        quantity.append(1)
                    else:
                        quantity.append(int(max2.group(1)))
                    items.append(max2.group(2))
                    sum1 = max2.group(3).replace(':','.')
                    prices.append((sum1))
                    sum1 = max2.group(3).replace(':','.')
                    prices_no_dollar.append(float(sum1))
        

subtotal= round(amount-tax, 2)

   
print("Tax: ")  
print(tax)
print("Amount:")
print(amount)
print("Subtotal:")
print(subtotal)
print("Items:")
print(items)
# print(prices)
print("Prices:")
print(prices_no_dollar)
print("Quantity:")
print(quantity)

Tax: 
4.18
Amount:
79.66
Subtotal:
75.48
Items:
['chinese buffet', 'soda', 'desserts']
Prices:
['51.96', 7.36, '15.56']
Quantity:
[4, 4, 4]
